In [1]:
import numpy as np
from random import randint
import pygame as py
import time

py.init()

(6, 0)

In [2]:
class Country:
    def __init__(self,population,chosen,childs,lucky_factor,mutation_factor,size,options):
        self.population = population
        self.chosen = chosen
        self.childs = childs
        self.lucky_factor = lucky_factor
        self.mutation_factor = mutation_factor
        self.size = size
        self.options = options
        self.Individuals = self.createFirstGeneration()
        
    def get_population(self):
        return self.Individuals
        
    def evolve(self,fitness):
        # Return best player from last gen
        self.Individuals,best = self.Selection(fitness)

        
        self.Individuals = self.breeding()
        self.mutation()            
        return self.Individuals,best
    
    def createFirstGeneration(self):
        self.Individuals = np.random.randint(self.options,size = self.size)
        for i in range(self.population-1):
            self.Individuals = np.vstack(
                (self.Individuals,np.random.randint(self.options,size = self.size)))
        return self.Individuals
    
    def Selection(self,fitness):
        inds = fitness.argsort()
        self.Individuals = self.Individuals[inds]
        Selected = np.array(self.Individuals[self.Individuals.shape[0]-1])
        for i in range(self.Individuals.shape[0]):
            if (i > self.Individuals.shape[0] - self.chosen):
                Selected = np.vstack((Selected,self.Individuals[i]))
            elif (np.random.uniform() <= self.lucky_factor):
                Selected = np.vstack((Selected,self.Individuals[i]))
        return Selected,self.Individuals[self.Individuals.shape[0]-1]
    
    def breeding(self):
        NextGen = np.array(self.createChild(self.Individuals[0],self.Individuals[1]))
        for i in range(0,self.Individuals.shape[0]-1):
            for j in range(self.childs):
                NextGen = np.vstack((NextGen,self.createChild(self.Individuals[i],self.Individuals[i+1])))
        return NextGen
    
    def createChild(self,parent1,parent2):
        child = np.array([])
        for i in range(parent1.shape[0]):
            if (randint(0,1) == 1):
                child = np.append(child,parent1[i])
            else:
                child = np.append(child,parent2[i])
        return child
    
    def mutation(self):
        for i in range(self.Individuals.shape[0]):
            if (np.random.uniform() <= self.mutation_factor):
                self.Individuals[i,randint(0,self.Individuals.shape[1]-1)] = np.random.randint(self.options)

In [3]:
class Game_Env:
    def __init__(self,rounds,land,p_per_l,price,players,render):
        self.rounds = rounds
        self.army = np.array([0,0,0,0])
        self.points = np.array([0,0,0,0])
        self.land = np.array([land,land,land,land])
        self.p_per_l = p_per_l
        self.price = price
        self.players = players
        self.render = render
        self.screen_size = (250 * players,400)
        self.colors = [[255,0,0],[255,255,0],[0,0,255],[0,255,255]]
        self.font = py.font.SysFont('Calibri', 25, False, False)
        if render:
            self.screen = py.display.set_mode(self.screen_size)
    
    def renderer(self):
        if self.render:
            start = 0
            for i in range(self.players):
                if (i != 0):
                    start = 50*self.land[i-1] + start
                py.draw.rect(self.screen,self.colors[i],
                         [start,0,self.land[i]*50,self.screen_size[1]],0)
                self.screen.blit(self.font.render(str(self.points[i]) + ' ' + str(self.army[i]),True,[0,0,0]),
                            [start+10, self.screen_size[1]/4])   
                py.display.flip()
                
    def play(self,Individual1,Individual2,Individual3,Individual4):
        Individual = np.vstack((Individual1,Individual2,Individual3,Individual4))
        self.army = np.array([0,0,0,0])
        for i in range(self.rounds):
            if self.render:
                self.renderer()
                time.sleep(0.5)
            self.points += self.land *self.p_per_l
            for j in range(Individual.shape[0]): 
                if (Individual[j,i] == 1 and self.points[j]>=self.price):
                    self.points[j] -= self.price
                    self.army[j] += 1
                elif (Individual[j,i] == 2):
                    defender = j + 1
                    if (defender >= self.players):
                        defender = 0
                    self.attack(j,defender)
                if (self.land[j] == 0):
                    self.points[j] = 0
        return self.points[0],self.points[1],self.points[2],self.points[3]
    
    def attack(self,attacker,defender):
        if (attacker == defender):
            return
        a_power = self.army[attacker]
        self.army[attacker] -= self.army[defender]
        self.army[defender] -= a_power
        if (self.army[defender] < 0):
            if (self.land[defender] <= 0):
                self.army[defender] = 0
                defender = defender + 1
                if (defender >= self.players):
                    defender = 0
                self.attack(attacker,defender)
            else:
                self.land[defender] -=1
                self.land[attacker] +=1
                self.army[defender] = 0
        elif (self.army[attacker] < 0):
            self.army[attacker] = 0

In [4]:
def Fitness(population1,population2,population3,population4):
    fitness1 = np.array([])
    fitness2 = np.array([])
    fitness3 = np.array([])
    fitness4 = np.array([])
    for i in range(population1.shape[0]):
        env = Game_Env(rounds,5,10,100,4,False)
        reward1, reward2, reward3, reward4 = env.play(population1[i],population2[i],population3[i],population4[i])
        fitness1 = np.append(fitness1,reward1)
        fitness2 = np.append(fitness2,reward2)
        fitness3 = np.append(fitness3,reward3)
        fitness4 = np.append(fitness4,reward4)
    return fitness1.astype(int),fitness2.astype(int),fitness3.astype(int),fitness4.astype(int)

In [5]:
Generations = 5000
population = 20
chosen = 5
childs = 4
lucky_factor = 0.0
mutation_factor = 0.3
rounds = 50
options = 3

Country1 = Country(population,chosen,childs,lucky_factor,mutation_factor,rounds,options)
Country2 = Country(population,chosen,childs,lucky_factor,mutation_factor,rounds,options)
Country3 = Country(population,chosen,childs,lucky_factor,mutation_factor,rounds,options)
Country4 = Country(population,chosen,childs,lucky_factor,mutation_factor,rounds,options)

Population1 = Country1.get_population()
Population2 = Country2.get_population()
Population3 = Country3.get_population()
Population4 = Country4.get_population()

In [6]:
for i in range(Generations):
    print ('Generations : ',i, ' Population : ',Population1.shape[0],end = '\r')
    
    fitness1,fitness2,fitness3,fitness4 = Fitness(Population1,Population2,Population3,Population4)
    
    r_win = np.argmax(np.vstack((fitness1,fitness2,fitness3,fitness4)),axis = 0)

    Population1,best1 = Country1.evolve(fitness1)
    Population2,best2 = Country2.evolve(fitness2)
    Population3,best3= Country3.evolve(fitness3)
    Population4,best4 = Country4.evolve(fitness4)

    
    if (i>4997):
        input()
        env = Game_Env(rounds,5,10,5,4,True)
        env.play(best1,best2,best3,best4)      

Generations :  4998  Population :  17
Generations :  4999  Population :  17


In [13]:
size = (1000,400)
screen = py.display.set_mode(size)

In [31]:
py.draw.rect(screen,[255,0,0],[0,0,250,400],0)
py.display.flip()

In [103]:
np.array([2,5,54,654]).argsort()

array([0, 1, 2, 3])

In [23]:
input()

''

In [10]:
env = Game_Env(rounds,5,10,5,4,True)
env.play(best1,best2,best3,best4)  

(1255, 4420, 3230, 0)